<a href="https://colab.research.google.com/github/SahputraS/Outbreak-Simulation-and-Detection-Testing/blob/main/Rio_Dengue_Risk_Classification_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---- base
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# ---- viz (keep only what you use)
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px

# ---- sklearn / imbalanced-learn / xgboost
from sklearn.model_selection import StratifiedKFold, GridSearchCV, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import (
    balanced_accuracy_score, make_scorer,
    accuracy_score, precision_score, recall_score, f1_score,
    average_precision_score, roc_auc_score,
    precision_recall_curve, confusion_matrix, classification_report,
)

from imblearn.pipeline import Pipeline           # must be imblearn's Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

from xgboost import XGBClassifier

# ---- other libs you referenced
from itertools import product
from tqdm.auto import tqdm
from statsmodels.tsa.stattools import ccf

# ---- tensorflow (only if you actually use it below)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
def get_data(ibge, ey_start, ey_end):
  url = "https://info.dengue.mat.br/api/alertcity"
  geocode = ibge
  disease = "dengue"
  format = "csv"
  ew_start = 1
  ew_end = 53
  ey_start = ey_start
  ey_end = ey_end

  params =(
      "&disease="
      + f"{disease}"
      + "&geocode="
      + f"{geocode}"
      + "&disease="
      + f"{disease}"
      + "&format="
      + f"{format}"
      + "&ew_start="
      + f"{ew_start}"
      + "&ew_end="
      + f"{ew_end}"
      + "&ey_start="
      + f"{ey_start}"
      + "&ey_end="
      + f"{ey_end}"
  )

  url_resp = "?".join([url, params])
  dados = pd.read_csv(url_resp, index_col='SE')
  return dados

In [ ]:
 # rio_neighbours_all = [3304557, 3302007, 3305554, 3303500,3301702, 3305109, 3300456, 3303203, 3302858, 3304144, 3302270, 3302858]
rio_neighbours = [3304557, 3302007, 3305554, 3303500, 3301702, 3305109, 3303203, 3302858]

In [ ]:
# geojson_url = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json"

# muni = gpd.read_file(geojson_url)
# muni["id"] = muni["id"].astype(str)

# rio = muni[muni["id"].str.startswith("330")].copy() # Municipality rio ibge starts with 330
# ribge = [str(x).zfill(7) for x in rio_neighbours]

# rpl = pd.DataFrame({"ibge": rio["id"]})
# rpl["selected"] = "other"
# rpl.loc[rpl["ibge"].isin(ribge), "selected"] = "neighbour"
# rpl.loc[rpl["ibge"] == "3304557", "selected"] = "rio_city"

# fig = px.choropleth(
#     rpl,
#     geojson=rio.__geo_interface__,
#     locations="ibge",
#     featureidkey="properties.id",
#     color="selected",
#     color_discrete_map={
#         "other":   "lightgrey",
#         "neighbour": "orange",
#         "rio_city": "maroon",
#     }
# )

# fig.update_traces(marker_line_width=0.4, marker_line_color="black")
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin=dict(l=0, r=0, t=0, b=0), coloraxis_showscale=False)

# fig.show()


In [ ]:
rj = []
for i, ibge in enumerate(rio_neighbours):
    data = get_data(ibge=ibge, ey_start=2010, ey_end=2025)
    data['region'] = f"r{i+1}"
    rj.append(data)

all = pd.concat(rj, ignore_index=True)

In [ ]:
all.columns

In [ ]:
all.head(5)

## Data Exploration

Because I want to learn how to predict the nivel, and the source uses
1. Weather
2. Twitter
3. Reported Cases

In [ ]:
all.columns

In [ ]:
cols = ['data_iniSE', 'nivel','p_inc100k', 'tweet', 'Rt', 'p_rt1', 'tempmin', 'umidmax', 'umidmed', 'umidmin', 'tempmed', 'tempmax', 'region']
all_filt = all[cols].copy()

In [ ]:
all_filt.isna().sum()

In [ ]:
def plot_ts(df, cols, date_col="data_iniSE", region_col="region", na_fill=0):

    df = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df[date_col]):
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    df[cols] = df[cols].fillna(na_fill)

    regions = df[region_col].unique()

    for col in cols:
        plt.figure(figsize=(17, 5))
        for r in regions:
            g = df[df[region_col] == r].sort_values(date_col)
            plt.plot(g[date_col], g[col], marker='o', markersize=2, label=f"Region {r}")
        plt.title(col)
        plt.xlabel("Date")
        plt.ylabel(col)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
vars_to_plot = ['p_inc100k','tweet','tempmin','umidmax','umidmed','umidmin','tempmed','tempmax']
plot_ts(all_filt, vars_to_plot, date_col="data_iniSE", region_col="region", na_fill=-5)

In [ ]:
## From the graph above, tweet data seems to exists for only the rio, therefore I will not include tweets
all_filt2 = all_filt.copy()
all_filt2.drop(columns=['tweet'], inplace=True)
all_filt2.head(5)

## Baseline Model 1
Tomorrow is the same as today

In [ ]:
data_b1 = all_filt2[['data_iniSE', 'nivel', 'region']].copy()
data_b1['pred'] = data_b1['nivel'].shift(-1)
data_b1 = data_b1[data_b1['data_iniSE'] != pd.to_datetime('2010-01-03')]
data_b1 = data_b1.dropna()

In [ ]:
data_b1.head(5)

In [ ]:
def eval(y_true, y_pred, region=None):
    acc = round(accuracy_score(y_true, y_pred), 3)
    precision = round(precision_score(y_true, y_pred, average="weighted", zero_division=0), 3)
    recall = round(recall_score(y_true, y_pred, average="weighted", zero_division=0), 3)
    f1 = round(f1_score(y_true, y_pred, average="weighted", zero_division=0), 3)

    row = pd.DataFrame([{
        "region": region,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }])
    return row

In [ ]:
reg = data_b1['region'].unique().tolist()
eval_baseline = pd.DataFrame()

for _, r in enumerate(reg):
    data = data_b1[data_b1['region']==r]
    eval_baseline = pd.concat([eval_baseline, eval(data['nivel'], data['pred'], r)], ignore_index=True)

In [ ]:
eval_baseline

## Data Preparation For XGBOOST

Transform the data to make the brazilian dataset to be more general (based on only incidence and weathers info) by:
- Making the pseudo Rt from the incidence cases (to be added)
- Change the nivel (alarm level) into 0/1 where 0 is flat or going down and 1 is if there is an increase in level
- Make the predictors are the average of some weeks

In [ ]:
def add_lags(df, cols, max_lag=4, include_lag0=False):

    df = df.sort_values(['region','data_iniSE']).copy()
    g = df.groupby('region', group_keys=False)
    for c in cols:
        if include_lag0:
            df[f"{c}_lag0"] = g[c].shift(0)
        for L in range(1, max_lag + 1):
            df[f"{c}_lag{L}"] = g[c].shift(L)
    return df

In [ ]:
def prep(df, h=1, train_frac=0.75, lag_cols=None, max_lag=4, include_lag0=False):

    df['data_iniSE'] = pd.to_datetime(df['data_iniSE'])
    df = df.sort_values(['region','data_iniSE'])

    # 1) Target: up=1 vs last week, then shift FORWARD by horizon h
    df['nivel_binary'] = df.groupby('region')['nivel'].diff().gt(0).astype(int).fillna(0)
    df['nivel_binary'] = df.groupby('region')['nivel_binary'].shift(-h)

    # 2) Drop unused raw cols if present
    for c in ['Rt','p_rt1','nivel']:
        if c in df.columns:
            df = df.drop(columns=c)

    # 3) Choose columns to lag (default: all numeric except id/date/target)
    if lag_cols is None:
        non_feat = {'data_iniSE','region','nivel_binary'}
        lag_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in non_feat]

    # 4) Add lags (past-only, no leakage)
    df = add_lags(df, lag_cols, max_lag=max_lag, include_lag0=include_lag0)

    # 5) Figure out which lag columns were added to enforce completeness
    added_lag_cols = []
    if include_lag0:
        added_lag_cols += [f"{c}_lag0" for c in lag_cols]
    added_lag_cols += [f"{c}_lag{L}" for c in lag_cols for L in range(1, max_lag+1)]

    # 6) Per-region chronological split; drop rows missing target/lag features
    regs = df['region'].unique()
    tr_parts, te_parts = [], []
    need_cols = ['nivel_binary'] + added_lag_cols

    for r in regs:
        gdf = df[df['region'] == r].copy()
        n = len(gdf)
        tr_end = int(np.floor(train_frac * n))
        print(f"region: {r}, total week: {n}, train until week: {tr_end}")

        g_tr = gdf.iloc[:tr_end].dropna(subset=need_cols)
        g_te = gdf.iloc[tr_end:].dropna(subset=need_cols)

        tr_parts.append(g_tr)
        te_parts.append(g_te)

    data_ml  = pd.concat(tr_parts, ignore_index=True).dropna()
    data_test = pd.concat(te_parts, ignore_index=True).dropna()
    return data_ml, data_test

In [ ]:
data_ml, data_test = prep(all_filt2)
data_ml.head(5)

Xy split

In [ ]:
NON_FEAT = {"data_iniSE", "region", "nivel_binary"}

def make_Xy(df):
    cols = [c for c in df.columns if c not in NON_FEAT]
    # cols = ['p_inc100k', 'tempmin', 'tempmin_lag1', 'p_inc100k_lag1']
    X = df[cols]
    y = df["nivel_binary"].astype(int)
    return X, y, cols

In [ ]:
X_train_raw, y_train, feat_cols = make_Xy(data_ml)
X_test_raw = data_test.reindex(columns=feat_cols)
y_test = data_test["nivel_binary"].astype(int)

## Class Imbalance - SMOT-EEN Sampling (Illustration Purposes)

In [ ]:
def plot_pie (y, title=""):
    vc = pd.Series(y).value_counts().sort_index()
    labels = [f"{cls} ({cnt}, {cnt/len(y):.1%})" for cls, cnt in vc.items()]
    plt.figure(figsize=(5,5))
    plt.pie(vc.values, labels=labels,  colors=["darkblue", "darkred"], startangle=90)
    plt.title(title)
    plt.axis("equal")
    plt.show()

In [ ]:
plot_pie(y_train, "Train label distribution (before SMOTE)")
plot_pie(y_test,  "Test label distribution")

SMOTE first selects a minority class instance a at random and finds its k nearest minority class neighbors. The synthetic instance is then created by choosing one of the k nearest neighbors b at random and connecting a and b to form a line segment in the feature space. The synthetic instances are generated as a convex combination of the two chosen instances a and b.

EEN (Edited Nearest Neighbors) is a "cleaning" mechanism.

By combining SMOTE with ENN, SMOTENN is able to generate synthetic samples that are more representative of the minority class and reduce the presence of noisy samples. This can lead to improved generalization performance of the model.

In [ ]:
# SMOTE-EEN Sampling
smenn = SMOTEENN()
X_train2, y_train2 = smenn.fit_resample (X_train_raw, y_train)

In [ ]:
plot_pie(y_train2, "Train label distribution (after SMOTE)")

## XGB Models

In [ ]:
select_param_xgb = XGBClassifier(
    objective="binary:logistic", eval_metric="aucpr",
    n_estimators=300, max_depth=3, learning_rate=0.06,
    subsample=0.9, colsample_bytree=0.9, random_state=42
)

In [ ]:
fin_xgb = XGBClassifier(
    objective="binary:logistic", eval_metric="aucpr",
    tree_method="hist", random_state=42
)

## Make Pipeline For Gridsearch

In [ ]:
pipe = Pipeline([
    ("scale",    StandardScaler()),
    ("smoteenn", SMOTEENN(smote=SMOTE(random_state=42), random_state=42)),
    ("select",   SelectFromModel(select_param_xgb, threshold="median")),
    ("xgb",      fin_xgb),
])

## Gridsearch

In [ ]:
param_grid = {
    # "smoteenn__sampling_strategy": [0.5, 0.7, 1.0],
    # "smoteenn__smote__k_neighbors": [3, 5],
    "select__threshold": ["median", "mean"],
    "xgb__n_estimators": [400, 800],
    "xgb__max_depth": [3, 4, 5],
    "xgb__learning_rate": [0.03, 0.06],
    "xgb__subsample": [0.8, 1.0],
    "xgb__colsample_bytree": [0.8, 1.0],
}

In [ ]:
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# gs = GridSearchCV(
#     estimator=pipe,
#     param_grid=param_grid,
#     #scoring via balanced acc due to imbalanced data
#     scoring=make_scorer(balanced_accuracy_score),
#     cv=cv,
#     n_jobs=-1,
#     refit=True,
#     verbose=0,
# )

In [ ]:
# gs.fit(X_train_raw, y_train)

# print("Best CV balanced accuracy:", gs.best_score_)
# print("Best params:", gs.best_params_)

In [ ]:
# best_model = gs.best_estimator_

In [ ]:
# selector    = best_model.named_steps["select"]

# mask = selector.get_support()

# if hasattr(X_train_raw, "columns"):
#     all_cols = np.asarray(X_train_raw.columns)
# else:
#     all_cols = np.asarray(feature_cols)

# selected_cols = all_cols[mask]
# dropped_cols  = all_cols[~mask]

# print(f"{mask.sum()} features kept / {len(all_cols)} total")
# print("Selected features:", selected_cols.tolist())

In [ ]:
horizon = 4
best_mod_list = []

for hh in range(horizon):

  print("Process horizon", hh+1)

  ## Make dataset
  data_ml, data_test = prep(all_filt2, h=hh+1)

  ## Split Xy
  X_train_raw, y_train, feat_cols = make_Xy(data_ml)
  X_test_raw = data_test.reindex(columns=feat_cols)
  y_test = data_test["nivel_binary"].astype(int)

  ## Do gridsearch per horizon
  gs.fit(X_train_raw, y_train)

  print("For horizon", hh+1, "best CV balanced accuracy:", gs.best_score_)
  print("For horizon", hh+1, "best params:", gs.best_params_)

  # Take best model
  best_model = gs.best_estimator_
  best_mod_list.append(best_model)

  selector    = best_model.named_steps["select"]

  mask = selector.get_support()

  if hasattr(X_train_raw, "columns"):
      all_cols = np.asarray(X_train_raw.columns)
  else:
      all_cols = np.asarray(feature_cols)

  selected_cols = all_cols[mask]
  dropped_cols  = all_cols[~mask]

  print(f"For horizon {hh+1}, {mask.sum()} features kept / {len(all_cols)} total")
  print("For horizon", hh+1, "selected features:", selected_cols.tolist())




In [ ]:
# Save the best models to a file
with open("best_models.txt", "w") as f:
    for i, model in enumerate(best_mod_list):
        f.write(f"--- Horizon {i+1} ---\n")
        f.write(str(model))
        f.write("\n\n")

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

shutil.copy("best_models.txt", "/content/drive/MyDrive/best_models.txt")
print("best_models.txt saved to Google Drive.")

## Test

In [ ]:
  ## Make dataset
  data_ml, data_test = prep(all_filt2, h=hh+1)

  ## Split Xy
  X_train_raw, y_train, feat_cols = make_Xy(data_ml)
  X_test_raw = data_test.reindex(columns=feat_cols)
  y_test = data_test["nivel_binary"].astype(int)


In [ ]:
y_pred_df = pd.DataFrame()

i = 0
for best_model in best_mod_list:

  print("Process horizon", i+1)

  ## Make dataset
  data_ml, data_test = prep(all_filt2, h=i+1) #fix later to streamline better
  i +=1

  ## Split Xy
  X_train_raw, y_train, feat_cols = make_Xy(data_ml)
  X_test_raw = data_test.reindex(columns=feat_cols)
  y_test = data_test["nivel_binary"].astype(int)

  y_pred = best_model.predict(X_test_raw)

  # Calculate evaluation metrics
  accuracy = balanced_accuracy_score(y_test, y_pred) #accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)

  print("XGBoost Balanced Accuracy:", round(accuracy,2))
  print("F1 Score:", round(f1,2))
  print("Recall:", round(recall,2))
  print("Precision:", round(precision,2))

  # PLot Heeatmap
  cm = confusion_matrix(y_test, y_pred)

  plt.figure(figsize=(8, 6))
  sns.set_theme(style="whitegrid")
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
  plt.title(f"Horizon {i}")
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  plt.show()